<a href="https://colab.research.google.com/github/WilliamAshbee/DataminingAssignments/blob/main/DM_A3_Ashbee.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import sklearn as sk
import pandas as pd
import numpy as np


In [2]:
#url = 'http://archive.ics.uci.edu/ml/machine-learning-databases/heart-disease/cleve.mod'
#df1 = pd.read_csv(url,header=None,skiprows=(i for i in range(20)),delim_whitespace=True,error_bad_lines=False)

##no longer using cleve mod since Dr. Zhang mentioned it was acceptable to use processed.cleveland.data


In [3]:
""" obsolete code for processing clevemod
pain = {'angina': 1, 'asympt': 4,'notang':2,'abnang':3}
gender = {'male':1,'fem':0}
bsl20 = {'true':1,'fal':0}
restingecg = {'norm':0,'abn':1, 'hyp':2}
exindang = {'true':1,'fal':0}
slope = {'up':1, 'flat':2, 'down':3}
thal = {'norm':1, 'fix':2, 'rev':3}
state= {'buff':1,'sick':-1}
code = {'H':0,'S1':1,'S2':2,'S3':3,'S4':4}
eleventh = {'?':-10000}
twelth = {'?':-10000}
#df2 = df1.replace({1: gender,2:pain,5:bsl20,6:restingecg,8:exindang,10:slope,11:eleventh,12:{**thal, **twelth},13:state,14:code})
"""

" obsolete code for processing clevemod\npain = {'angina': 1, 'asympt': 4,'notang':2,'abnang':3}\ngender = {'male':1,'fem':0}\nbsl20 = {'true':1,'fal':0}\nrestingecg = {'norm':0,'abn':1, 'hyp':2}\nexindang = {'true':1,'fal':0}\nslope = {'up':1, 'flat':2, 'down':3}\nthal = {'norm':1, 'fix':2, 'rev':3}\nstate= {'buff':1,'sick':-1}\ncode = {'H':0,'S1':1,'S2':2,'S3':3,'S4':4}\neleventh = {'?':-10000}\ntwelth = {'?':-10000}\n#df2 = df1.replace({1: gender,2:pain,5:bsl20,6:restingecg,8:exindang,10:slope,11:eleventh,12:{**thal, **twelth},13:state,14:code})\n"

Links that helped in my solution

https://stackoverflow.com/questions/17114904/python-pandas-replacing-strings-in-dataframe-with-numbers
\
https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.replace.html

In [4]:
url2 = 'http://archive.ics.uci.edu/ml/machine-learning-databases/heart-disease/processed.cleveland.data'
df_clev = pd.read_csv(url2,header=None,error_bad_lines=False)

In [5]:
questionmark = {'?':'-10000'}

dfcn = df_clev
dfcn = dfcn.replace({11:questionmark,12:questionmark})
dfcn = dfcn.to_numpy().astype(float)

print(np.sum(dfcn=='?'))#,np.sum(df2n=='?'))

0


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:7: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  import sys


In [6]:
import sklearn as sk
from sklearn.linear_model import LogisticRegression
from sklearn import preprocessing

import os
y = dfcn[:,13]

X = dfcn[:,:13]

scaler = preprocessing.StandardScaler().fit(X)
X = scaler.fit_transform(X)



Another link that led to my solutions \
https://medium.com/@rrfd/boosting-bagging-and-stacking-ensemble-methods-with-sklearn-and-mlens-a455c0c982de
\
https://scikit-learn.org/stable/auto_examples/model_selection/plot_roc_crossval.html
\
https://medium.com/@plog397/auc-roc-curve-scoring-function-for-multi-class-classification-9822871a6659

In [14]:
#note multiclass roc curve is a little poorly defined, so i went with
#solution from: https://medium.com/@plog397/auc-roc-curve-scoring-function-for-multi-class-classification-9822871a6659
#runtime on google colab was about 4 minutes
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import plot_roc_curve
import matplotlib.pyplot as plt
from sklearn import preprocessing
from sklearn.metrics import roc_auc_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import BaggingClassifier
from sklearn import svm
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import VotingClassifier
from sklearn.metrics import f1_score


cv = StratifiedKFold(n_splits=10)
y = y.squeeze()

#code multiclass_roc_auc_score from medium article above and below
#https://medium.com/@plog397/auc-roc-curve-scoring-function-for-multi-class-classification-9822871a6659
def multiclass_roc_auc_score(y_test, y_pred, average="macro"):
  lb = preprocessing.LabelBinarizer()
  lb.fit(y_test)
  y_test = lb.transform(y_test)
  y_pred = lb.transform(y_pred)
  return roc_auc_score(y_test, y_pred, average=average)

import random
def getMonteCarloSearchParams():#random search is faster than grid search for most purposes
  C = random.choice(np.logspace(-6, -1, 10).tolist())
  gamma = random.choice([0.001, 0.01, 0.1, 1])
  k = random.choice([3,4,5,6,7])
  d = random.choice([3,4,5,6,7])
  maxit = random.choice([10**3,10**4,10**5,10**6])

  return C,gamma,k,d,maxit

epochs = 10

scores = np.zeros((epochs,10))
aucs = np.zeros((epochs,10))
f1_scores = np.zeros((epochs,10))

params = np.zeros((epochs,5))


for e in range(epochs):
  C,gamma,k,d,maxit = getMonteCarloSearchParams()
  params[e,0] = C
  params[e,1] = gamma
  params[e,2] = k
  params[e,3] = d
  params[e,4] = maxit
  
  print('\nc',C,'gamma',gamma,'k',k,'d',d,'maxit',maxit)
  rfc = RandomForestClassifier(max_depth=d)
  knnc = KNeighborsClassifier(n_neighbors=k)
  bagging_knnc = BaggingClassifier(knnc, max_samples=0.4, max_features=4)
  lrc = LogisticRegression(max_iter = maxit, solver='lbfgs', multi_class='auto')
  svc = svm.SVC(kernel = 'rbf',gamma = gamma,max_iter = maxit,C= C,decision_function_shape='ovo', probability=True)
  adac = AdaBoostClassifier(base_estimator=svc)
  vc = VotingClassifier(estimators=[("logistic regression",lrc),('Random Forests', rfc), ('KNeighbors', bagging_knnc), ('adaSVCrbf', adac)], voting='hard')
  classifier = vc
  
  
  for i, (train, test) in enumerate(cv.split(X, y)):
    print('\nepoch',e,' cv of 10:',i+1)
    classifier.fit(X[train], y[train])
    y_pred = classifier.predict(X[test])

    scores[e,i] = classifier.score(X[test],y[test])
    aucs[e,i] = multiclass_roc_auc_score(y[test],y_pred)
    f1_scores[e,i] = f1_score(y[test], y_pred, average='macro')
    print("score",'{:.2f}'.format(scores[e,i]))
    print('auc score','{:.2f}'.format(aucs[e,i]))
    print('f1 score', '{:.2f}'.format(f1_scores[e,i]))




c 0.0005994842503189409 gamma 1 k 6 d 4 maxit 100000

epoch 0  cv of 10: 1
score 0.58
auc score 0.53
f1 score 0.20

epoch 0  cv of 10: 2
score 0.58
auc score 0.55
f1 score 0.22

epoch 0  cv of 10: 3
score 0.55
auc score 0.51
f1 score 0.15

epoch 0  cv of 10: 4
score 0.60
auc score 0.55
f1 score 0.23

epoch 0  cv of 10: 5
score 0.57
auc score 0.54
f1 score 0.25

epoch 0  cv of 10: 6
score 0.57
auc score 0.54
f1 score 0.22

epoch 0  cv of 10: 7
score 0.53
auc score 0.50
f1 score 0.14

epoch 0  cv of 10: 8
score 0.57
auc score 0.53
f1 score 0.22

epoch 0  cv of 10: 9
score 0.57
auc score 0.56
f1 score 0.28

epoch 0  cv of 10: 10
score 0.53
auc score 0.51
f1 score 0.15

c 3.5938136638046257e-06 gamma 0.001 k 6 d 5 maxit 10000

epoch 1  cv of 10: 1
score 0.61
auc score 0.58
f1 score 0.27

epoch 1  cv of 10: 2
score 0.55
auc score 0.51
f1 score 0.15

epoch 1  cv of 10: 3
score 0.55
auc score 0.51
f1 score 0.15

epoch 1  cv of 10: 4
score 0.60
auc score 0.55
f1 score 0.25

epoch 1  cv of 10:

In [15]:
#"C,gamma,k,d,maxit"
sc_par = params[np.argmax(np.mean(scores,axis=1)),:]
auc_par = params[np.argmax(np.mean(aucs,axis=1)),:]
f1_par = params[np.argmax(np.mean(f1_scores,axis=1)),:]

 

print('best average scores cross validation value:',"{:.2f}".format(np.max(np.mean(scores,axis =1 ))))
print('model params C,gamma,k,d,maxit:','{:e} {:6g} {:d} {:d} {:d} '.format(sc_par[0],sc_par[1],(int)(sc_par[2]),(int)(sc_par[3]),(int)(sc_par[4])))

print('\nbest average auc value:',"{:.2f}".format(np.max(np.mean(aucs,axis =1 ))))
print('model params C,gamma,k,d,maxit:','{:e} {:6g} {:d} {:d} {:d} '.format(auc_par[0],auc_par[1],(int)(auc_par[2]),(int)(auc_par[3]),(int)(auc_par[4])))

print('\nbest average f1score value:',"{:.2f}".format(np.max(np.mean(f1_scores,axis =1 ))))
print('model params C,gamma,k,d,maxit:','{:e} {:6g} {:d} {:d} {:d} '.format(f1_par[0],f1_par[1],(int)(f1_par[2]),(int)(f1_par[3]),(int)(f1_par[4])))


best average scores cross validation value: 0.57
model params C,gamma,k,d,maxit: 2.782559e-02      1 5 6 1000000 

best average auc value: 0.54
model params C,gamma,k,d,maxit: 7.742637e-03   0.01 3 6 10000 

best average f1score value: 0.23
model params C,gamma,k,d,maxit: 7.742637e-03   0.01 3 6 10000 


In [16]:
#https://www.kaggle.com/iamhungundji/covid19-symptoms-checker
#bonus work dataset: predict severity
#see bonus ipython notebook

#Begin bonus section